###**Deep Learning Model to predict mRNA degrdation**

##Introduction


In [2]:
import json
import pandas as pd
import numpy as np

#visualization
import plotly.express as px

#deep learning libraries
import tensorflow.keras.layers as L
import tensorflow as tf
from sklearn.model_selection import train_test_split

 #setting seeds
tf.random.set_seed(2023)
np.random.seed(2023)


###Import data from Kaggle


In [3]:
!pip install --upgrade --force-rein --no-deps kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kaggle: filename=kaggle-1.5.13-py3-none-any.whl size=77733 sha256=0f37a5f3a28290a7d8e03186054f5580e7f7025ad3641b1077d31bf7dc0de816
  Stored in directory: /root/.cache/pip/wheels/9c/45/15/6d6d116cd2539fb8f450d64b0aee4a480e5366bb11b42ac763
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.13
    Uninstalling kaggle-1.5.13:
      Successfully uninstalled kaggle-1.5.13


In [4]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"vaishnavekv","key":"bceac5310d5a0d48b1a8b3a2b42759ed"}'}

In [5]:
!mkdir ~/.kaggle

In [6]:
!cp kaggle.json ~/.kaggle/

In [7]:
!chmod 600 ~/.kaggle/kaggle.json

In [8]:
!kaggle competitions download -c stanford-covid-vaccine

 90% 81.0M/89.8M [00:01<00:00, 91.7MB/s]
100% 89.8M/89.8M [00:01<00:00, 89.7MB/s]


In [9]:
!unzip /content/stanford-covid-vaccine.zip -d /content/data

Streaming output truncated to the last 5000 lines.
  inflating: /content/data/bpps/id_33920944d.npy  
  inflating: /content/data/bpps/id_3397e6fde.npy  
  inflating: /content/data/bpps/id_33b33ce4e.npy  
  inflating: /content/data/bpps/id_33b754424.npy  
  inflating: /content/data/bpps/id_33bd903ba.npy  
  inflating: /content/data/bpps/id_33cbe6509.npy  
  inflating: /content/data/bpps/id_33cfa91dc.npy  
  inflating: /content/data/bpps/id_33d4dc6d0.npy  
  inflating: /content/data/bpps/id_33e8cdd16.npy  
  inflating: /content/data/bpps/id_33ea0b43b.npy  
  inflating: /content/data/bpps/id_33f606ec5.npy  
  inflating: /content/data/bpps/id_3406dc4f8.npy  
  inflating: /content/data/bpps/id_340d5ef56.npy  
  inflating: /content/data/bpps/id_34201072a.npy  
  inflating: /content/data/bpps/id_34202b68b.npy  
  inflating: /content/data/bpps/id_34286a9d3.npy  
  inflating: /content/data/bpps/id_34340f0bf.npy  
  inflating: /content/data/bpps/id_344128dc3.npy  
  inflating: /content/data/bpps

##Exploring Dataset

In [10]:
train = pd.read_json("/content/data/train.json", lines= True)
test = pd.read_json("/content/data/test.json", lines= True)
sample_df = pd.read_csv("/content/data/sample_submission.csv")

target_cols = ["reactivity", "deg_Mg_pH10","deg_Mg_50C","deg_pH10","deg_50C"]
Model_Train = True

In [11]:
train.head(5)

,index,id,sequence,structure,predicted_loop_type,signal_to_noise,SN_filter,seq_length,seq_scored,reactivity_error,deg_error_Mg_pH10,deg_error_pH10,deg_error_Mg_50C,deg_error_50C,reactivity,deg_Mg_pH10,deg_pH10,deg_Mg_50C,deg_50C
0,0,id_001f94081,GGAAAAGCUCUAAUAACAGGAGACUAGGACUACGUAUUUCUAGGUA...,.....((((((.......)))).)).((.....((..((((((......,EEEEESSSSSSHHHHHHHSSSSBSSXSSIIIIISSIISSSSSSHHH...,6.894,1,107,68,"[0.1359, 0.20700000000000002, 0.1633, 0.1452, ...","[0.26130000000000003, 0.38420000000000004, 0.1...","[0.2631, 0.28600000000000003, 0.0964, 0.1574, ...","[0.1501, 0.275, 0.0947, 0.18660000000000002, 0...","[0.2167, 0.34750000000000003, 0.188, 0.2124, 0...","[0.3297, 1.5693000000000001, 1.1227, 0.8686, 0...","[0.7556, 2.983, 0.2526, 1.3789, 0.637600000000...","[2.3375, 3.5060000000000002, 0.3008, 1.0108, 0...","[0.35810000000000003, 2.9683, 0.2589, 1.4552, ...","[0.6382, 3.4773, 0.9988, 1.3228, 0.78770000000..."
1,1,id_0049f53ba,GGAAAAAGCGCGCGCGGUUAGCGCGCGCUUUUGCGCGCGCUGUACC...,.....(((((((((((((((((((((((....)))))))))).)))...,EEEEESSSSSSSSSSSSSSSSSSSSSSSHHHHSSSSSSSSSSBSSS...,0.193,0,107,68,"[2.8272, 2.8272, 2.8272, 4.7343, 2.5676, 2.567...","[73705.3985, 73705.3985, 73705.3985, 73705.398...","[10.1986, 9.2418, 5.0933, 5.0933, 5.0933, 5.09...","[16.6174, 13.868, 8.1968, 8.1968, 8.1968, 8.19...","[15.4857, 7.9596, 13.3957, 5.8777, 5.8777, 5.8...","[0.0, 0.0, 0.0, 2.2965, 0.0, 0.0, 0.0, 0.0, 0....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[4.947, 4.4523, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[4.8511, 4.0426, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[7.6692, 0.0, 10.9561, 0.0, 0.0, 0.0, 0.0, 0.0..."
2,2,id_006f36f57,GGAAAGUGCUCAGAUAAGCUAAGCUCGAAUAGCAAUCGAAUAGAAU...,.....((((.((.....((((.(((.....)))..((((......)...,EEEEESSSSISSIIIIISSSSMSSSHHHHHSSSMMSSSSHHHHHHS...,8.800,1,107,68,"[0.0931, 0.13290000000000002, 0.11280000000000...","[0.1365, 0.2237, 0.1812, 0.1333, 0.1148, 0.160...","[0.17020000000000002, 0.178, 0.111, 0.091, 0.0...","[0.1033, 0.1464, 0.1126, 0.09620000000000001, ...","[0.14980000000000002, 0.1761, 0.1517, 0.116700...","[0.44820000000000004, 1.4822, 1.1819, 0.743400...","[0.2504, 1.4021, 0.9804, 0.49670000000000003, ...","[2.243, 2.9361, 1.0553, 0.721, 0.6396000000000...","[0.5163, 1.6823000000000001, 1.0426, 0.7902, 0...","[0.9501000000000001, 1.7974999999999999, 1.499..."
3,3,id_0082d463b,GGAAAAGCGCGCGCGCGCGCGCGAAAAAGCGCGCGCGCGCGCGCGC...,......((((((((((((((((......))))))))))))))))((...,EEEEEESSSSSSSSSSSSSSSSHHHHHHSSSSSSSSSSSSSSSSSS...,0.104,0,107,68,"[3.5229, 6.0748, 3.0374, 3.0374, 3.0374, 3.037...","[73705.3985, 73705.3985, 73705.3985, 73705.398...","[11.8007, 12.7566, 5.7733, 5.7733, 5.7733, 5.7...","[121286.7181, 121286.7182, 121286.7181, 121286...","[15.3995, 8.1124, 7.7824, 7.7824, 7.7824, 7.78...","[0.0, 2.2399, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....","[0.0, -0.5083, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...","[3.4248, 6.8128, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.0, -0.8365, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...","[7.6692, -1.3223, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
4,4,id_0087940f4,GGAAAAUAUAUAAUAUAUUAUAUAAAUAUAUUAUAGAAGUAUAAUA...,.....(((((((.((((((((((((.(((((((((....)))))))...,EEEEESSSSSSSBSSSSSSSSSSSSBSSSSSSSSSHHHHSSSSSSS...,0.423,0,107,68,"[1.665, 2.1728, 2.0041, 1.2405, 0.620200000000...","[4.2139, 3.9637000000000002, 3.2467, 2.4716, 1...","[3.0942, 3.015, 2.1212, 2.0552, 0.881500000000...","[2.6717, 2.4818, 1.9919, 2.5484999999999998, 1...","[1.3285, 3.6173, 1.3057, 1.3021, 1.1507, 1.150...","[0.8267, 2.6577, 2.8481, 0.40090000000000003, ...","[2.1058, 3.138, 2.5437000000000003, 1.0932, 0....","[4.7366, 4.6243, 1.2068, 1.1538, 0.0, 0.0, 0.7...","[2.2052, 1.7947000000000002, 0.7457, 3.1233, 0...","[0.0, 5.1198, -0.3551, -0.3518, 0.0, 0.0, 0.0,..."


In [12]:
train.shape

(2400, 19)

In [13]:
test.shape

(3634, 7)

In [14]:
test.head()

,index,id,sequence,structure,predicted_loop_type,seq_length,seq_scored
0,0,id_00073f8be,GGAAAAGUACGACUUGAGUACGGAAAACGUACCAACUCGAUUAAAA...,......((((((((((.(((((.....))))))))((((((((......,EEEEEESSSSSSSSSSBSSSSSHHHHHSSSSSSSSSSSSSSSSHHH...,107,68
1,1,id_000ae4237,GGAAACGGGUUCCGCGGAUUGCUGCUAAUAAGAGUAAUCUCUAAAU...,.....((((..((((((...(((((.....((((....)))).......,EEEEESSSSIISSSSSSIIISSSSSIIIIISSSSHHHHSSSSIIII...,130,91
2,2,id_00131c573,GGAAAACAAAACGGCCUGGAAGACGAAGGAAUUCGGCGCGAAGGCC...,...........((.(((.(.(..((..((..((((...))))..))...,EEEEEEEEEEESSISSSISISIISSIISSIISSSSHHHSSSSIISS...,107,68
3,3,id_00181fd34,GGAAAGGAUCUCUAUCGAAGGAUAGAGAUCGCUCGCGACGGCACGA...,......((((((((((....))))))))))((((((..((.(((.....,EEEEEESSSSSSSSSSHHHHSSSSSSSSSSSSSSSSIISSISSSHH...,107,68
4,4,id_0020473f7,GGAAACCCGCCCGCGCCCGCCCGCGCUGCUGCCGUGCCUCCUCUCC...,.....(((((((((((((((((((((((((((((((((((((((((...,EEEEESSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSS...,130,91


In [15]:
train = train.query("signal_to_noise >=1")

In [16]:
fig = px.histogram(test, "seq_length", nbins = 30, title = "Sequence length Distribution", width = 1000,height=500)
fig.show()

##Preprocessing of Dataset

In [17]:
public_df = test.query("seq_length ==107")
private_df = test.query("seq_length == 130")

In [18]:
token2int = {x: i for i, x in enumerate("().ACGUBEHIMSX")}
token2int

{'(': 0,
 ')': 1,
 '.': 2,
 'A': 3,
 'C': 4,
 'G': 5,
 'U': 6,
 'B': 7,
 'E': 8,
 'H': 9,
 'I': 10,
 'M': 11,
 'S': 12,
 'X': 13}

In [19]:
def dataframe_to_array(df):
  return np.transpose(np.array(df.values.tolist()), (0,2,1))

In [20]:
def dataframe_label_encoding(df, token2int, cols=["sequence", "structure","predicted_loop_type"]):
  return dataframe_to_array(df[cols].applymap(lambda seq: [token2int[x] for x in seq]))

In [21]:
train_inputs = dataframe_label_encoding(train, token2int)
train_labels = dataframe_to_array(train[target_cols])

In [22]:
public_inputes = dataframe_label_encoding(public_df,token2int)

private_inputes = dataframe_label_encoding(private_df, token2int)

##Building the model

In [23]:
x_train, x_val,y_train,y_val = train_test_split(train_inputs, train_labels,
                                                test_size=0.1,random_state=42,
                                                stratify=train.SN_filter)

In [24]:
def MCRMSE(y_true, y_pred):
  colwise_mse = tf.reduce_mean(tf.square(y_true-y_pred),axis=1)
  return tf.reduce_mean(tf.sqrt(colwise_mse),axis =1)

In [25]:
def build_model(embed_size,
                seq_len=107,
                pred_len=68,
                dropout=0.5,
                sp_dropout=0.2,
                embed_dim=200,
                hidden_dim=256):
  inputs = L.Input(shape=(seq_len,3))
  embed = L.Embedding(input_dim=embed_size, output_dim=embed_dim)(inputs)
  reshape = tf.reshape(embed, shape=(-1,embed.shape[1],embed.shape[2]*embed.shape[3]))
  
  hidden = L.SpatialDropout1D(sp_dropout)(reshape)

  hidden = L.Bidirectional(
    L.GRU(
      hidden_dim,
      dropout=dropout,
      return_sequences = True,
      kernel_initializer = "orthogonal",))(hidden)
  hidden = L.Bidirectional(
    L.GRU(
      hidden_dim,
      dropout=dropout,
      return_sequences = True,
      kernel_initializer = "orthogonal",))(hidden)
  hidden = L.Bidirectional(
    L.GRU(
      hidden_dim,
      dropout=dropout,
      return_sequences = True,
      kernel_initializer = "orthogonal",))(hidden)
  
  truncated = hidden[:, :pred_len]
  out = L.Dense(5, activation = "linear")(truncated)  
  model = tf.keras.Model(inputs = inputs, outputs = out)
  model.compile(optimizer="Adam",loss=MCRMSE)
  return model 

In [26]:
model = build_model(embed_size=len(token2int))
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 107, 3)]          0         
                                                                 
 embedding (Embedding)       (None, 107, 3, 200)       2800      
                                                                 
 tf.reshape (TFOpLambda)     (None, 107, 600)          0         
                                                                 
 spatial_dropout1d (SpatialD  (None, 107, 600)         0         
 ropout1D)                                                       
                                                                 
 bidirectional (Bidirectiona  (None, 107, 512)         1317888   
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 107, 512)         118272

##Training the model

In [27]:
import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR)

In [28]:
if Model_Train:
   history = model.fit(
       x_train,
       y_train,
       validation_data=(x_val,y_val),
       batch_size = 64,
       epochs = 40,
       verbose = 2,
       callbacks=[tf.keras.callbacks.ReduceLROnPlateau(patience=5),
                  tf.keras.callbacks.ModelCheckpoint("Model/model.hs")])  

Epoch 1/40
30/30 - 60s - loss: 0.4555 - val_loss: 0.3899 - lr: 0.0010 - 60s/epoch - 2s/step
Epoch 2/40
30/30 - 37s - loss: 0.3843 - val_loss: 0.3642 - lr: 0.0010 - 37s/epoch - 1s/step
Epoch 3/40
30/30 - 36s - loss: 0.3636 - val_loss: 0.3484 - lr: 0.0010 - 36s/epoch - 1s/step
Epoch 4/40
30/30 - 37s - loss: 0.3483 - val_loss: 0.3387 - lr: 0.0010 - 37s/epoch - 1s/step
Epoch 5/40
30/30 - 35s - loss: 0.3380 - val_loss: 0.3258 - lr: 0.0010 - 35s/epoch - 1s/step
Epoch 6/40
30/30 - 36s - loss: 0.3287 - val_loss: 0.3183 - lr: 0.0010 - 36s/epoch - 1s/step
Epoch 7/40
30/30 - 34s - loss: 0.3224 - val_loss: 0.3112 - lr: 0.0010 - 34s/epoch - 1s/step
Epoch 8/40
30/30 - 34s - loss: 0.3149 - val_loss: 0.3014 - lr: 0.0010 - 34s/epoch - 1s/step
Epoch 9/40
30/30 - 36s - loss: 0.3072 - val_loss: 0.2965 - lr: 0.0010 - 36s/epoch - 1s/step
Epoch 10/40
30/30 - 35s - loss: 0.3025 - val_loss: 0.2836 - lr: 0.0010 - 35s/epoch - 1s/step
Epoch 11/40
30/30 - 35s - loss: 0.2918 - val_loss: 0.2749 - lr: 0.0010 - 35s/ep

##Evaluating the model

In [29]:
if Model_Train:
  fig = px.line(history.history,
                y = ["loss","val_loss"],
                labels = {"index": "epoch", "value": "MCRMSE"},
                title = "history")
  fig.show()

In [71]:
model_public= build_model(seq_len = 107, pred_len = 107, embed_size = len(token2int))
model_private= build_model(seq_len = 130, pred_len = 130, embed_size = len(token2int))

model_public.load_weights("Model/model.hs")
model_private.load_weights("Model/model.hs")

In [82]:
public_preds = model_public.predict(public_inputes)
private_preds = model_private.predict(private_inputes)

private_preds.shape

94/94 [==============================] - 2s 24ms/step


(3005, 130, 5)

In [83]:
pred_ls = []

for df,preds in [(public_df,public_preds), (private_df,private_preds)]:
  for i,uid in enumerate(df.id):
    single_pred = preds[i]
    single_df = pd.DataFrame(single_pred,columns = target_cols)
    single_df["id_seqpos"] = [f"{uid}_{x}" for x in range(single_df.shape[0])]
    pred_ls.append(single_df)
  
preds_df = pd.concat(pred_ls)

In [84]:
preds_df.head()

,reactivity,deg_Mg_pH10,deg_Mg_50C,deg_pH10,deg_50C,id_seqpos
0,0.548773,0.589042,0.487342,1.995412,0.750711,id_00073f8be_0
1,1.865386,2.885223,3.123976,4.071233,2.759126,id_00073f8be_1
2,1.391194,0.575726,0.584777,0.551595,0.695919,id_00073f8be_2
3,1.204477,1.126149,1.640303,1.123501,1.700925,id_00073f8be_3
4,0.790026,0.568052,0.787452,0.475916,0.844788,id_00073f8be_4
